In [1]:
import pandas as pd

In [59]:
master=pd.read_csv('consolidated_master.csv')
#build master draft data
draft_file_dir = 'Draft Results/'
draft_data = []
for season in master[master.season>=2012].season.drop_duplicates():
    try:
        draft_df = pd.read_csv(f'{draft_file_dir}{season}.csv')
    except:
        draft_df = pd.read_excel(f'{draft_file_dir}{season}.xlsx')
    if 'Year' not in draft_df.columns:
        draft_df['Year'] = season
    if 'Actual' not in draft_df.columns:
        draft_df['Actual']=''
    if draft_df[['Player','Position']].isnull().sum().sum() > 0:
        print(f'null values found in {season}')
        break
    draft_data.append(draft_df[['Year','Owner','Player','Position','Price','Actual']])

draft_cols_dict = {
    'DST':'DEF',
    'D':'DEF',
    'Def':'DEF'
}
full_draft_df = pd.concat(draft_data)
#full_draft_df=full_draft_df[full_draft_df.Year<2021] #compare to my old numbers
standardized_positions = []
for pos in full_draft_df.Position:
    if pos in draft_cols_dict.keys():
        standardized_positions.append(draft_cols_dict[pos])
    else:
        standardized_positions.append(pos)
full_draft_df['Position'] = standardized_positions
player_name_dict = {}
lkup_player = pd.read_csv('lkup_player.csv')
for index, row in lkup_player.iterrows():
    conc = ''.join([row['Name'],row['Position']])
    if conc not in player_name_dict.keys():
        player_name_dict[conc] = row['Standard Name']

#now add standard name to draft df
standard_players = []
for index, row in full_draft_df.iterrows():
    try:
        player_conc = ''.join([row['Player'],row['Position']])
    except:
        print(row['Player'], row['Position'])
    try:
        standard_players.append(player_name_dict[player_conc])
    except:
        standard_players.append(row['Player'])

full_draft_df['standard_player'] = standard_players

standard_player_concs = []
for index, row in full_draft_df.iterrows():
    standard_player_conc = ''.join([row['standard_player'],row['Position']])
    standard_player_concs.append(standard_player_conc)
full_draft_df['standard_player_conc'] = standard_player_concs

#dictoinary to map defense ids
def_dict = {
    'arizona':'100022',
    'atlanta':'100001',
    'baltimore':'100033',
    'buffalo':'100002',
    'carolina':'100029',
    'chicago':'100003',
    'cincinnati':'100004',
    'cleveland':'100005',
    'dallas':'100006',
    'denver':'100007',
    'detroit':'100008',
    'green-bay':'100009',
    'houston':'100034',
    'indianapolis':'100011',
    'jacksonville':'100030',
    'kansas-city':'100012',
    'la-chargers':'100024',
    'la-rams':'100014',
    'st-louis':'5239',
    'las-vegas':'100013',
    'miami':'100015',
    'minnesota':'100016',
    'new-england':'100017',
    'new-orleans':'100018',
    #ny-giants
    'ny-jets':'100020',
    'philadelphia':'100021',
    'pittsburgh':'100023',
    'san-francisco':'100025',
    'seattle':'100026',
    'tampa-bay':'100027',
    #tennessee
    'washington':'100028'
}

inv_def_dict = {v: k for k, v in def_dict.items()} #probably should stick with the city names

names_dict = pd.read_csv('names_dict.csv')
player_id_dict = {}
for index, row in names_dict.iterrows():
    player_id_dict[row['player_conc']] = row['player_id']

#add player_id column to draft data based on standard player name + position conc
full_draft_df['player_id'] = full_draft_df.standard_player_conc.map(player_id_dict)
full_draft_df.fillna('',inplace=True)
cleaned_player_ids=[]
for i in full_draft_df.player_id:
    try:
        clean_id = str(int(i))
        if clean_id in inv_def_dict.keys():
            clean_id = inv_def_dict[clean_id]
        cleaned_player_ids.append(clean_id)
    except:
        cleaned_player_ids.append('')
full_draft_df['player_id'] = cleaned_player_ids

position_ranks = pd.read_csv('position_ranks_thru_2023.csv')
player_ids = []
for url in position_ranks.player_url:
    split = url.split('/')
    if len(split[-1])>0:
        player_ids.append(str(split[-1]))
    else:
        player_ids.append(str(split[-2]))
position_ranks['player_id'] = player_ids

#get the actual position rank by position, which is the number we really need
position_ranks.sort_values(by=['season','position','position_rank'],ascending=[True,True,True],inplace=True)
actuals =[]
actual_rank=0
#previous_season=position_ranks.iloc[0].season
previous_position=position_ranks.iloc[0].position
for index, row in position_ranks.iterrows():
    if row['position']==previous_position:
        actual_rank=actual_rank+1
    else:
        actual_rank=1
    actuals.append(actual_rank)
    previous_position=row['position']
position_ranks['actual_pos_rank']=actuals

#if this is >0 then some cleanup is required, see notes below for example scenarios
if full_draft_df[~full_draft_df.player_id.isin(list(position_ranks['player_id']))].shape[0] > 0:
    print('There is a problem with missing player_ids in draft_df and/or position_ranks')

#notes
#so what we need to do is append the position ranks into the draft_df.  we hope to do this using player_id.  
#however some do not have player_id for whatever reason.  Some of these can be cleaned, probably by updating standard player names
#but not all.  so I am wondering if we could lkup based on conc of year + name + position?  
#still problem may be that the player simply does not exist in position ranks, which would therefore require manual cleanup 

#FINALLY create position_rank column for full_draft_df - ugh this could have probably been done more efficiently
position_rank_values= []
for index, row in full_draft_df.iterrows():
    if row['Actual'] =='':
        #these are position rank values by yahoo which are questionable...I do not know how they get them but they are different than mine
        position_rank = position_ranks[(position_ranks.player_id == row['player_id']) & (position_ranks.season == row['Year'])].actual_pos_rank.iloc[0]
    else:
        #we prefer this value because it was manually done by me
        position_rank = row['Actual']
    position_rank_values.append(position_rank)

full_draft_df['position_rank'] = position_rank_values

#convert Price to string
int_prices=[]
for price in full_draft_df.Price:
    if type(price)==str:
        price=int(price.replace(',',''))
    int_prices.append(price)
full_draft_df['Price'] = int_prices
full_draft_df.sort_values(by=['Year','Position','Price'],ascending=[True,True,False],inplace=True)
seasons=full_draft_df.Year.drop_duplicates()
season_dfs=[]
for season in seasons:
    position_spend_ranks=[]
    season_draft_df = full_draft_df[full_draft_df.Year==season]
    previous_position=season_draft_df.Position.iloc[0]
    previous_price=season_draft_df.Price.iloc[0]
    previous_spend_rank=1
    player_num=1
    for index, row in season_draft_df.iterrows():
        current_position = row['Position']
        current_price = row['Price']
        if current_position==previous_position:
            if current_price==previous_price:
                current_spend_rank = previous_spend_rank
            else:
                current_spend_rank = player_num
        else:
            current_spend_rank=1
            player_num = 1
        player_num=player_num+1
        position_spend_ranks.append(current_spend_rank)
        previous_position=current_position
        previous_price=current_price
        previous_spend_rank=current_spend_rank
    season_draft_df['spend_rank'] = position_spend_ranks

    #now handle position ranks
    normalized_position_ranks=[]
    season_draft_df.sort_values(by=['Position','position_rank'],ascending=[True,True],inplace=True)
    previous_position=season_draft_df.Position.iloc[0]
    previous_pos_rank=season_draft_df.position_rank.iloc[0]
    previous_pos_rank_norm=1
    for index, row in season_draft_df.iterrows():
        current_position = row['Position']
        current_pos_rank=row['position_rank']
        if current_position==previous_position:
            if current_pos_rank==previous_pos_rank:
                current_pos_rank_norm = previous_pos_rank_norm
            else:
                current_pos_rank_norm = previous_pos_rank_norm+1
        else:
            current_pos_rank_norm = 1
        normalized_position_ranks.append(current_pos_rank_norm)
        previous_position=current_position
        previous_pos_rank=current_pos_rank
        previous_pos_rank_norm=current_pos_rank_norm
    season_draft_df['position_rank_normalized'] = normalized_position_ranks
    season_dfs.append(season_draft_df)

#get a dictionary storing count of positions in starting roster for each season, so we can use this to calculcate appropriate bonus points
position_count_dict = {}
for season in full_draft_df.Year.drop_duplicates():
    #print(season)
    matchups_df=pd.read_csv(f'{season}_pre_matchups.csv')
    matchups_df['position']=matchups_df['position'].replace(' ','')
    matchup=matchups_df[matchups_df.matchup_url==matchups_df.matchup_url.iloc[0]]

    position_counts=matchup.position.value_counts()
    position_dict = {}
    for index, row in pd.DataFrame(position_counts).iterrows():
        position_dict[index] = row['position']
    if 'Q/WR/T' in position_dict:
        position_dict['QB'] = position_dict['QB'] + position_dict['Q/WR/T']
        del position_dict['Q/WR/T']
    position_count_dict[season]=position_dict

full_seasons_draft_df=pd.concat(season_dfs)
full_seasons_draft_df['differential'] = full_seasons_draft_df.spend_rank - full_seasons_draft_df.position_rank_normalized
# full_seasons_draft_df['Position']=full_seasons_draft_df.Position.replace(" ",'')
bonus_points=[]
for index, row in full_seasons_draft_df.iterrows():
    number_managers = full_seasons_draft_df[full_seasons_draft_df.Year==row['Year']].Owner.drop_duplicates().shape[0]
    max_bonus = position_count_dict[row['Year']][row['Position']] * number_managers
    bonus=max_bonus-row['position_rank']+1
    if bonus<0:
        bonus=0
    bonus_points.append(bonus)

full_seasons_draft_df['top_pos_bonus'] =bonus_points
full_seasons_draft_df['draft_score'] = full_seasons_draft_df['differential'] + full_seasons_draft_df['top_pos_bonus']

full_seasons_draft_df

C:\Users\dunca\AppData\Local\Temp\ipykernel_7580\3636306414.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_draft_df['spend_rank'] = position_spend_ranks
C:\Users\dunca\AppData\Local\Temp\ipykernel_7580\3636306414.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_draft_df.sort_values(by=['Position','position_rank'],ascending=[True,True],inplace=True)
C:\Users\dunca\AppData\Local\Temp\ipykernel_7580\3636306414.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

,Year,Owner,Player,Position,Price,Actual,standard_player,standard_player_conc,player_id,position_rank,spend_rank,position_rank_normalized,differential,top_pos_bonus,draft_score
0,2012,Duncan,Bears Defense,DEF,21000,1,Chicago,ChicagoDEF,chicago,1.0,3,1,2,8.0,10.0
62,2012,Mark,49ers Defense,DEF,350000,6,San Francisco,San FranciscoDEF,san-francisco,6.0,1,2,-1,3.0,2.0
1,2012,Luke,Houston D,DEF,10000,7,Houston,HoustonDEF,houston,7.0,5,3,2,2.0,4.0
3,2012,Krista,Packers,DEF,1000,14,Green Bay,Green BayDEF,green-bay,14.0,6,4,2,0.0,2.0
4,2012,Bryan,Lions Defense,DEF,73000,29,Detroit,DetroitDEF,detroit,29.0,2,5,-3,0.0,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,2023,Bryan,Allen Lazard,WR,1,,Allen Lazard,Allen LazardWR,31268,98.0,44,47,-3,0.0,-3.0
8,2023,Krista,Skyy Moore,WR,3,,Skyy Moore,Skyy MooreWR,34009,107.0,38,48,-10,0.0,-10.0
19,2023,Scott,Mike Williams,WR,8,,Mike Williams (LAC),Mike Williams (LAC)WR,30120,110.0,25,49,-24,0.0,-24.0
69,2023,Luke,Kadarius Toney,WR,6,,Kadarius Toney,Kadarius ToneyWR,33408,114.0,30,50,-20,0.0,-20.0


In [61]:
full_seasons_draft_df.to_csv('full_seasons_draft_df.csv',index=False)

In [62]:
for season in full_seasons_draft_df.Year.drop_duplicates():
    season_draft=full_seasons_draft_df[full_seasons_draft_df.Year==season]
    avg_score=season_draft.draft_score.sum() / season_draft.shape[0]
    print(f'{season}: {avg_score}')

2012: 4.697916666666667
2013: 3.83
2014: 5.1138211382113825
2015: 5.513761467889908
2016: 5.417322834645669
2017: 5.503875968992248
2018: 6.375
2019: 6.048275862068966
2020: 5.787234042553192
2021: 5.946969696969697
2022: 6.525547445255475
2023: 6.649253731343284
